In [ ]:


#Get the path of the current working directory
import os
cwd = os.getcwd()
print(cwd)

from src.data_loader import DataLoader
from src.enumerations import *

# Instantiate the data loader

# Load the data
data_loader = DataLoader()

In [ ]:
from src.model_time import TimeHorizon
# Instantiate the TimeHorizon class
time_horizon = TimeHorizon(start_year=2015, end_year=2300, data_timestep=5, timestep=1)

# th = time_horizon.get_model_time_horizon()
#print(th)
print(time_horizon.data_time_horizon)  
# Pass the time_horizon object to other parts of the model

In [ ]:
path_to_justice = '/Users/palokbiswas/Desktop/pollockdevis_git/JUSTICE'

# import sys
# sys.path.insert(0, path_to_justice)



from src.economy.neoclassical import NeoclassicalEconomyModel
from src.emissions.emission import OutputToEmissions
#from economy.neoclassical import NeoclassicalEconomyModel

In [ ]:
model = NeoclassicalEconomyModel(input_dataset = data_loader, time_horizon=time_horizon)

In [ ]:
emissions = OutputToEmissions(input_dataset=data_loader, time_horizon=time_horizon)


In [ ]:
gdp = NeoclassicalEconomyModel.__getattribute__(model, 'gdp_dict')
#gdp = data_loader.GDP_DICT
testdf = gdp['SSP1']

population = NeoclassicalEconomyModel.__getattribute__(model, 'population_dict')
testdf2 = population['SSP1']

In [ ]:
print(model.__getattribute__('capital_elasticity_in_production_function'))

In [ ]:
print(model.__getattribute__('depreciation_rate_capital'))

In [ ]:
print(model.get_optimal_long_run_savings_rate())

In [ ]:
import numpy as np
#Create a fixed savings rate
fixed_savings_rate = np.copy(data_loader.SAVING_RATE_INIT_ARRAY).reshape(-1, 1)
#fixed_savings_rate Validated with RICE50 for timestep 1 and 5

set_year = time_horizon.model_time_horizon


for i, years in enumerate(set_year):
    
    t = i+1 #index starts at 0, so add 1 to get the year

    if t != 1: # no need to repeat for the first year

        next_rate = data_loader.SAVING_RATE_INIT_ARRAY + (model.get_optimal_long_run_savings_rate() - data_loader.SAVING_RATE_INIT_ARRAY)*((t - 1)/(len(set_year) - 1))
        # append to the fixed savings rate array for each year
        fixed_savings_rate = np.column_stack((fixed_savings_rate, next_rate))

from matplotlib import pyplot as plt
plt.plot(np.sum(fixed_savings_rate, axis=0))
plt.title("Fixed Savings Rate")
plt.xlabel("Time")
plt.ylabel("Savings Rate")
plt.show()

In [ ]:
len(set_year)

In [ ]:
scenario = SSP.SSP1
print(scenario.value)

for i in range(len(set_year)):
    output = model.run(scenario = scenario.value, timestep=i, savings_rate=fixed_savings_rate[:, i])
    #print shape of output
    print(output.shape)
    print(i)
    #should emissions be an array of shape region, timesteps?
    emis = emissions.run_emissions(timestep=i, scenario=scenario.value, output=output[:,i], emission_control_rate=0.0)


#run_emissions(self, timestep, scenario, output, emission_control_rate)

In [ ]:
print(output)

In [ ]:
import pandas as pd
testdf = pd.DataFrame(output)

## Start here for experimentation

In [1]:
from src.model_time import TimeHorizon
# Instantiate the TimeHorizon class
time_horizon = TimeHorizon(start_year=2015, end_year=2300, data_timestep=5, timestep=1)

# th = time_horizon.get_model_time_horizon()
#print(th)
# print(time_horizon.data_time_horizon)  

from src.climate.coupled_fair import CoupledFAIR

model = CoupledFAIR()

#model.fair_justice_run_init(time_horizon=time_horizon, scenarios=["ssp245"])

In [2]:
# # model.fair_fill_data(time_horizon=time_horizon, scenarios=["ssp245"])
# # model.test_init_fair(scenarios=["ssp245"], time_horizon=time_horizon)
# model.species_configs
# model.configs
from src.enumerations import *
scenario = SSP.SSP1
fair_scenario = [(ssp_to_rcp[scenario]).value]



print(fair_scenario)

['ssp119']


In [3]:
model.fair_justice_run_init(time_horizon=time_horizon, scenarios=fair_scenario)

In [4]:
#create a list of years 1750 to 2014 with 1 year timestep
#0 -> 1750
#265 -> 2015
#549 -> 2299
#FAIR takes up until 549 
#FAIR outputs t+1 for input of t

import time
import numpy as np
# justice_run_years = np.arange(
#     self.start_year_justice,
#     (self.end_year_fair + self.timestep_justice),
#     self.timestep_justice,
# )
justice_run_years = np.arange( #This generates till 2300
    2015,
    (2300 + 1),
    1,
)

#Loop through the number of years starting with zero for 1750
#0
#Need to do this prerun in the init
#Then figure out how to clear emissions and refill with new emissions



In [5]:
temperature = model.get_temperature_array()

temp = temperature[:,0,:,0]

In [6]:
loop_counter =  model.__getattribute__("fair_historical_timestep_run_count")
print(loop_counter)

265


In [7]:
len(justice_run_years)


286

In [7]:
for i in range(loop_counter, (((len(justice_run_years))+loop_counter)-1)):
    #print(i)
    model.stepwise_run(i)

In [11]:
"""
#This is for FAIR #Loop should run till (len(justice_run_years) -1 )
for i in range(loop_counter, (len(justice_run_years) - 1 )): # This works. The justice loop should run from loop counter and end with the last year of the justice run
    print(i , justice_run_years[i])
    # print(i)
    # start_time = time.time()
    model.stepwise_run(i)
    # end_time = time.time()  # palok
    # elapsed_time = end_time - start_time  # calculate the difference
    # f.calculate_toa_ocean_airborne_fraction()
    # f.prepare_output_xarrays()


"""

'\n#This is for FAIR #Loop should run till (len(justice_run_years) -1 )\nfor i in range(loop_counter, (len(justice_run_years) - 1 )): # This works. The justice loop should run from loop counter and end with the last year of the justice run\n    print(i , justice_run_years[i])\n    # print(i)\n    # start_time = time.time()\n    model.stepwise_run(i)\n    # end_time = time.time()  # palok\n    # elapsed_time = end_time - start_time  # calculate the difference\n    # f.calculate_toa_ocean_airborne_fraction()\n    # f.prepare_output_xarrays()\n\n\n'

In [8]:
temperature = model.get_temperature_array()

temp = temperature[:,0,:,0]

In [4]:
#Shape: timepoints, scenarios and ensemblemodels
current_emissions = model.emissions.sel(specie="CO2 FFI", scenario=fair_scenario)


In [5]:
#Plots the current emissions
current_emissions = current_emissions[:,0,:]


In [ ]:
# #create a list of years 1750 to 2014 with 1 year timestep
# import time
# import numpy as np



# #Loop through the number of years starting with zero for 1750
# loop_counter = 0
# #Need to do this prerun in the init
# #Then figure out how to clear emissions and refill with new emissions
# for i in range(0, len(years_fair)): #This should be 1 for justice (i think)
#     loop_counter = i
    
#     start_time = time.time()
#     model.stepwise_run(i)
#     end_time = time.time()  # palok
#     elapsed_time = end_time - start_time  # calculate the difference
#     # f.calculate_toa_ocean_airborne_fraction()
#     # f.prepare_output_xarrays()
#     #print(f"One iteration {i} of the loop took {elapsed_time} seconds to run.")

In [5]:
# temperature = model.get_temperature_array()

# temp = temperature[:,0,:,0]